# **Financial Outcomes of Private Equity Portfolio Companies post-IPO** <a id='top'></a>

## **Abstract**
This paper investigates the long-term financial outcomes of companies that were formally private equity-backed by assessing the financial performance of these companies post-private equity exit. Private equity companies rely mostly on private-to-private sale, private-to-public acquisition, or IPO to exit from their investments. Since the two former exit strategies do not always require public disclosure of financials upon exit, I rely on companies that IPO'd from 2010 through 2018 to base my analysis on. The analysis is broken into two sections. The first analysis, which utilizes an OLS regression, compares the growth rates of company financial metrics to companies that IPO'd during the same time frame that were not private equity-backed. The second analysis is an event study that seeks to find if private equity-backed companies experience similar growth rates post-IPO as they were prior to exit.
Below is all the source code for the two analyses that I run in the paper along with some of the outputs. 


**Click [here](#analysis_one) to jump to Analysis 1 <br>
**Click [here](#analysis_two) to jump to Analysis 2

<a id='data_cleaning'></a>
## **Data Cleaning and Initialization**

In [ ]:
# Run this cell to set up your notebook

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns 
import csv
import numpy as np
import pandas as pd
from psmpy.plotting import *
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy import stats
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from statistics import stdev
import pingouin as pg
import collections
import statsmodels.api as sm

%matplotlib inline
sns.set()
sns.set_context("talk")

from IPython.display import display, Latex, Markdown
pd.set_option('display.max_columns', 10)
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
# Importing Company Deal Info as well as company financials
deal_info = pd.read_csv(r'./company_list.csv')
company_fin = pd.read_csv(r'./company_financials_average_month_data.csv')

In [ ]:
# Fill all empty company financials with column median
for col in company_fin.columns[2:]:
    company_fin[col] = company_fin[col].apply(pd.to_numeric, errors='coerce')
    company_fin[col] = company_fin[col].fillna(company_fin[col].median())


For data cleaning purposes, I fill all NaN values in the dataset with median values of each column. 

Some of the columns must also be converted into numeric types. When the csv files are read, the cell data is sometimes inferred as "object" types when they should be numeric types instead.

In [ ]:
# Fill deal_info columns with median values
col = ['Year Founded', 'Revenue',
       'Gross Profit', 'Net Income', 'EBITDA', 'EBIT', 'Total Debt',
       'Fiscal Year', 'Valuation/Ebitda', 'Valuation/EBIT',
       'Valuation/Net Income', 'Valuation/Revenue', 'Valuation / Cash Flow',
       'Deal Size/EBITDA', 'Deal Size/EBIT', 'Deal Size/Net Income',
       'Deal Size/Revenue', 'Deal Size/Cash Flow',
       'Employees', 'Total Invested Equity','Deal Size', 'Deal Year']

for c in col:
    deal_info[c] = deal_info[c].apply(pd.to_numeric, errors='coerce')
    deal_info[c] = deal_info[c].fillna(deal_info[c].median())

In [ ]:
# cleaning all nan values and dropping first column of dataset
cleaned_fin = company_fin.fillna(0).drop(index = 0)
deal_info = deal_info.fillna(0)

deal_info["EBITDA Margin %"] = deal_info["EBITDA Margin %"].str.replace("%","")

deal_info["Company"] = deal_info["Company"].astype(str)
deal_info["EBITDA Margin %"] = deal_info["EBITDA Margin %"].astype(float)
deal_info["Employees"] = deal_info["Employees"].astype(int) 
deal_info["Total Invested Equity"] = deal_info["Total Invested Equity"].astype(float) 
deal_info = deal_info.fillna(0)


cleaned_fin.rename(columns={"Company Year":"Company", '    Revenue % Growth':'revenue_growth', 
                            '    EBITDA % Growth':'ebitda_growth', 'Net Income Growth':'net_income_growth', 
                         'ROE % Growth':'roe_growth', 'Liquidity % Growth':'liquidity_growth'}, inplace = True)
                         
cleaned_fin["Company"] = cleaned_fin["Company"].astype(str)

cleaned_fin.sort_values("Company", inplace=True)
cleaned_fin.columns

After that, we can calculate how far each financial statement was published from the time of IPO. This gives up information as to how the companies are doing per year in relation to IPO date.

In [ ]:
deal_info = deal_info[deal_info["Company"].isin(cleaned_fin["Company"])]
cleaned_fin = cleaned_fin[cleaned_fin["Company"].isin(deal_info["Company"])]

deal_info[deal_info["Primary Industry Sector"] == "Consumer Products and Services (B2C)"]

# keep track of how many years company has been around for
years_around = []

for fiscal,founded in zip(deal_info["Deal Year"], deal_info["Year Founded"]):
    years = (fiscal - founded) if (fiscal > 0 and founded > 0) else 0
    years_around.append(years)

deal_info["Years Since Founding"] = years_around
deal_info["Years Since Founding"] = deal_info["Years Since Founding"].astype(int)

Next we need to clean the data such that the quarterly financials are being used effectively. I lose around 20,000 rows by factoring out and just keeping the december quarterly trends. The issue with this is that by doing so, I am potentially losing out on best performance months for companies that may historically suffer during Q4 of the year. For instance, consumer companies would tend to do better when compared to, say banks, in winter months as consumer spending increases in the holidays.

In [ ]:
# we label the PE-treated firms with binary labels
def sum_and_avg_company_years(cleaned_fin, aggregate_mappings):
    _agg_fin = cleaned_fin
    display(cleaned_fin['Date of Financials'])
    # First change all company financial months into their respective years
    _agg_fin["Fin Year"] = _agg_fin["Date of Financials"].str.replace(r"-\w*", "", regex=True).astype(int) + 2000

    _rel_fin = cleaned_fin[cleaned_fin["Date of Financials"].str.startswith("-Dec")]
    _rel_fin["Fin Year"] = _rel_fin["Date of Financials"].str.replace("-Dec", "").astype(int) + 2000
    _rel_fin = _rel_fin[['Company', 'Fin Year', 'revenue_growth',
                         'ebitda_growth', 'net_income_growth', 
                         'roe_growth', 'liquidity_growth']]
    
        
    #Group by relative financial year and then average the data to account for seasonality and lack of quarters
    #For the time being, we will also keep growth estimates for only december growth patterns
    _agg_fin = _agg_fin.groupby(["Company", "Fin Year"]).agg(aggregate_mappings).reset_index()

    _agg_fin.merge(_rel_fin, on=['Company', 'Fin Year'])
    return _agg_fin

#Dictionary telling us which columns we should keep from the data that we have on hand
agg = {'    Total Revenue':np.mean,
       'revenue_growth':np.mean, 
       '    Gross Profit':np.mean,
       '    Total Operating Profit/(Loss)':np.mean, 
       '    EBITDA':np.mean,
       '    EBITDA (Normalized)':np.mean,
       '    EBITDA Margin':np.mean, 
       'ebitda_growth':np.mean,
       '    Net Income (Normalized)':np.mean,
       'net_income_growth':np.mean,
       '    Net Income (Analyst Normalized)':np.mean,
       '    Diluted EPS':np.mean,
       '    Diluted Weighted Average Shares Outstanding':np.mean,
       '    Total Current Assets':np.mean, 
       '    Net Property, Plant and Equipment':np.mean,
       '    Total Non-Current Assets':np.mean, 
       '    Total Assets':np.mean,
       '    Total Current Liabilities':np.mean, 
       '    Total Non-Current Liabilities':np.mean,
       '    Total Liabilities':np.mean, 
       '    Total Equity':np.mean,
       'ROE':np.mean,  
       'roe_growth':np.mean,
       '    Equity Attributable to Parent Stockholders':np.mean,
       '    Total Debt':np.mean,
       '    Total Shares Outstanding (TSO)':np.mean,
       '    Working Capital':np.mean,
       '    Change in Cash':np.mean,
       '    Capital Expenditure (Calc)':np.mean,
       '    Issuance of/(Payments for) Common Stock, Net':np.mean,
       '    Cash Dividends Paid':np.mean,
       '    Cash and Cash Equivalents, Beginning of Period':np.mean,
       '    Cash and Cash Equivalents, End of Period':np.mean,
       'liquidity_growth':np.mean,
       '    Net Income from Continuing Operations Sequential % Growth':np.mean,
       '    Net Income Available to Common Stockholders Sequential % Growth':np.mean,
       '    Current Ratio':np.mean,
       '    Quick Ratio':np.mean,
       '    Debt to Equity':np.mean,
       '    Total Debt to Equity':np.mean,
       '    Total Asset Turnover':np.mean,
       '    Normalized Return on Equity':np.mean,
       '    Normalized Return on Assets':np.mean,
       '    Normalized Return on Invested Capital':np.mean,
       '    Stock Price':np.mean,
       '    Market Cap':np.mean,
       '    EV':np.mean,
       '    EV to Revenue':np.mean,
       '    EV to EBIT':np.mean,
       '    EV to EBIT (Normalized)':np.mean,
       '    EV to EBIT (Analyst Normalized)':np.mean,
       '    EV to EBITDA':np.mean,
       '    EV to EBITDA (Normalized)':np.mean,
       '    EV to EBITDA (Analyst Normalized)':np.mean,
       '    Price to Earnings':np.mean,
       '    Price to Earnings (Normalized)':np.mean,
       '    Price to Earnings (Analyst Normalized)':np.mean,
       '    Forward Price to Earnings':np.mean,
       '    Price to Book (PB)':np.mean,
       '    Price to Cash Flow (PCF)':np.mean,
       '    Price to Tangible Book Value':np.mean}

_rel_fin = sum_and_avg_company_years(cleaned_fin, agg)

Outliers that are either very high or very low in the dataset are also removed. Some of the companies in the dataset have growth values that are incredibly large (think 100000%) and as a result are heavily skewing the results of the analysis.

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    
    if fence_low == np.nan or fence_high == np.nan or fence_high == fence_low:
        return df_in
    else:
        df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    
    return df_out

def rmv_all_outliers(df_list):
    cleaned_list = []
    for df in df_list:
        for col in df.columns:
            if is_numeric_dtype(df[col]):
                df = remove_outlier(df, col)
        cleaned_list.append(df)
    return cleaned_list

<a id='analysis_one'></a>
## **Analysis 1 - Propensity Score Estimation**

Through my analysis I investigate differences in revenue, EBITDA, net income, liquidity, and ROE growth rates of PE-treated and non-treated companies at one-, three-, and five-years after IPO. I find no statistical significance in revenue growth, net income growth, and ROE growth. However, I do find statistical significance in both liquidity and EBITDA growth. These results go in hand with the business model of private equity firms. Firms raise as much debt as possible when acquiring businesses, and thus the cash at hand (liquidity) that these companies will have to pay off debt is directly impacted by the amount of debt they take on. For each growth measure of growth tested, I compile my results in the form of an attached table and violin plot to demonstrate the distribution of data between treated and control groups.

In [ ]:
category = ["Primary Industry Sector"]

data_with_categ = pd.concat([deal_info.drop(columns=category),
                            pd.get_dummies(deal_info[category], columns=category, drop_first=False)], axis=1)
data_with_categ.shape

Merging deal information with company financial information at IPO gives us more variables to use to calculate propensity scores.

In [ ]:
data_with_categ_merged = data_with_categ.merge(_rel_fin, left_on=['Company', 'Fiscal Year'], 
                                        right_on=['Company', 'Fin Year'], how="left")
data_with_categ_merged.shape

One of the one-hot-encode data columns are dropped since we are fitting the intercept.

Propensity Scores are then calculated using the columns below with a Logistic Regression model. 

In [ ]:
T = 'PE'

X_axis = ['Revenue', 'Gross Profit', 'Net Income', 'EBITDA', 'EBIT', 'Total Debt',
       'Valuation/Ebitda', 'Valuation/EBIT', 'Valuation/Net Income',
       'Valuation/Revenue', 'Valuation / Cash Flow', 'EBITDA Margin %',
       'Employees', 'Years Since Founding',
       'Primary Industry Sector_Business Products and Services (B2B)',
       'Primary Industry Sector_Consumer Products and Services (B2C)',
       'Primary Industry Sector_Energy',
       'Primary Industry Sector_Financial Services',
       'Primary Industry Sector_Healthcare',
       'Primary Industry Sector_Information Technology',]
# 'Primary Industry Sector_Materials and Resources' column dropped
ps_model = LogisticRegression(C=1e6, fit_intercept=True).fit(data_with_categ_merged[X_axis], data_with_categ_merged[T])
data_ps = data_with_categ_merged.assign(propensity_score=ps_model.predict_proba(data_with_categ_merged[X_axis])[:, 1])
# print(data_ps["propensity_score"]) # print to show propensity scores calculated

The covariance table below helps to show us that there is not obvious multicollinearity in the model. 

In other words, none of the covariances are close to +/-1.

In [ ]:
df = data_ps[X_axis]
df[X_axis[:11]].cov()

As can be seen below, the weights of the data are not representative of the total sample size, but are importantly similar to each other. This would indicate that all least some of the confounders used to relatively compare portfolio and non-treated companies were well-captured in the model.

After this, I graph the distribution of propensity scores for both groups of company data. What I get as a result is significant overlap between propensity scores of treated and non-treated companies. The distributions are also *relatively* centered close to 0.5.

In [ ]:
#Weights of the data
weight_t = 1/data_ps.query("PE==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("PE==0")["propensity_score"])
print("Original sample size:", deal_info.shape[0])
print("Treated pop. sample size:", sum(weight_t))
print("Unreated pop. sample size:", sum(weight_nt))

In [ ]:
# data_ps = data_ps[data_ps["propensity_score"] > 0.01]

less_than_1 = data_ps.query("PE==1")["propensity_score"] 

less_than_0 = data_ps.query("PE==0")["propensity_score"] 

sns.distplot(less_than_0, kde=False, label="Non Treated")
sns.distplot(less_than_1, kde=False, label="Treated")
plt.legend()
plt.title("Propensity Scores");

Once we calculate the propensity scores, we then append the scores to each of the company financials by mapping company name to propensity scores. 

Additionally, we also write in all the financial information if the financials came from a company with PE treatment or not.

PE = 1 means the company is a portfolio company.

In [ ]:
pe_firms = deal_info.query("PE==1")
names = pe_firms['Company'].unique()
treatment = []
for val in _rel_fin['Company']:
    t = 1 if val in names else 0
    treatment.append(t)
_rel_fin["PE"] = treatment
# _rel_fin #print to show with new PE column 

In [ ]:
# make dictionary of prop scores mapping name to score
_rel_fin['propensity_score'] = _rel_fin['Company']

new_map = {}
for index, deal in data_ps.iterrows():
        comp = deal["Company"]
        if comp not in new_map:
            new_map[comp] = deal["propensity_score"]
            

In [ ]:
_rel_fin["propensity_score"] = _rel_fin["propensity_score"].replace(new_map)
_rel_fin = _rel_fin[_rel_fin["Company"].isin(new_map.keys())]

_rel_fin["propensity_score"] = _rel_fin["propensity_score"].astype(float)
# _rel_fin #print to show with new propensity score mapping

The functions below are useful to determine the year of a company financial and how many years before or after the IPO it was published.

In [ ]:
# defines function to return relevant financial of the company per year
def fin_by_year(df, company_name, year_after_deal):
    year = df[df['Company'] == company_name]['Deal Year'].values[0] + year_after_deal
    return _rel_fin[(_rel_fin['Company'] == company_name) & (_rel_fin['Fin Year'] == year)]

def x_years_out(df, years):
    names = df['Company'].unique()
    df_orig = df
    x_year_data = []
    for company in names:
        df = fin_by_year(df_orig, company, years)
        x_year_data.append(df)
    df_out = pd.concat(x_year_data)
    return df_out

By knowing how many years out our financials are from the date of IPO, we can infer the results of our model for the years that we care about.
                                                                                                            
In the model, years 1, 3, and 5 are the years of interest.

In [ ]:
data_results = []
for years in [1,3,5]:
    x_year = x_years_out(data_ps, years)
    data_results.append(x_year)

# Making a function that does the same thing below to not mess up anything
def data_time_horizon(dataset, list_of_years):
    data_results = []
    for years in list_of_years:
        x_year = x_years_out(dataset, years)
        data_results.append(x_year)
    return data_results
    
# data_results[0] #uncomment to show financials from year 1

Now we can build an OLS model for each of the years and for each of the growth metrics of interest.

**Y** in this case are the variables that we want to analyze.

**X** in this model are the PE treatment columns and the propensity scores that we calculated previously.

In [ ]:
for df in data_results:
    var_to_analyze = 'liquidity_growth'
    df = remove_outlier(df, var_to_analyze)
    Y = df[var_to_analyze].values #scale the variables
    X = df[["PE","propensity_score"]].values

    model = sm.OLS(Y,X)
    results = model.fit()
    display(results.params)
    print(results.t_test([1, 0]))
    print("Number of Observations: ", Y.size)

Now that the model works, we can build one for each of the years and each of the variables of interest.

In [ ]:
vars_to_analyze = ['revenue_growth','ebitda_growth','net_income_growth','liquidity_growth','roe_growth']
for var_to_analyze in vars_to_analyze:
    models = []
    t_tests = []
    year = 1;
    for df in data_results:
        df = remove_outlier(df, var_to_analyze)
        Y = df[var_to_analyze]
        X = df[["PE","propensity_score"]]
        X['Y'] = Y
        mod = smf.ols('Y ~ propensity_score + PE', data=X)
        res = mod.fit()
        t_test = res.t_test('PE = 0')
        print(f"\nYear {year} results for {var_to_analyze}")
        display(t_test)
        t_tests.append(t_test)
        models.append(res)
    # stargazer = Stargazer(models)             #USED TO OUTPUT THE MODEL IN LATEX FORMAT
    # display(Latex(stargazer.render_latex()))  #USED TO OUTPUT THE MODEL IN LATEX FORMAT

Output of the growth estimates is also graphed to see the overall distributions of both groups. 

Comment out the variables that you don't want to run and add variables that you do want to see distributions for in the vars_to_analyze list. Please only run one variable at a time.

In [ ]:
# Comment out and uncomment the variables that you'd like to display to compare the against the two 
# Please run only one variable at a time
vars_to_analyze = ['liquidity_growth'] #, 'roe_growth','revenue_growth','ebitda_growth','net_income_growth','liquidity_growth']

# Outputs a violin plot for both treated and non-treated companies per variable
# passed into to model
def show_var_distributions(vars_to_analyze:list[str], data_results:pd.DataFrame) -> None:
    pe = []
    no_pe = []
    for var_to_analyze in vars_to_analyze:
        count = 0
        for i in [1,3,5]:
            df = data_results[count]
            df = remove_outlier(df, var_to_analyze)
            Y = df[var_to_analyze] * 100
            X = df[["PE","propensity_score"]]
            X['Y'] = Y
            x_1 = X.query("PE==1")['Y']
            x_1 = x_1[x_1 > 0]
            x_1 = (x_1.values)
            
            x_pe = x_npe = np.full(len(x_1), 1, dtype=str)
            arr = np.full(len(x_1), i, dtype=str)
            data = {"Year":arr,"Value":x_1,"PE":x_pe}
            
            x_0 = X.query("PE==0")['Y']
            x_0 = x_0[x_0 > 0]
            x_0 = (x_0.values)
            x_npe = np.full(len(x_0), 0, dtype=str)
            arr_o = np.full(len(x_0), i, dtype=str)
            dato = {"Year":arr_o,"Value":x_0,"PE":x_npe}
            
            do = pd.DataFrame(dato)
            
            df = pd.DataFrame(data)
            
            pe.append(pd.concat([df,do], axis=0))
            
            count +=1
    df = pd.concat(pe, axis=0)
    sns.set(rc={'figure.figsize':(11.7,8.27)}, font_scale=2)
    ax = sns.violinplot(df, x = "Year", y="Value", hue='PE', split=True)
    ax.set(ylabel='Value (%)')
    ax.axhline(0, ls='--')
    plt.show()

show_var_distributions(vars_to_analyze, data_results)

<a id='analysis_two'></a>
## **Analysis 2 - Financial Performance Event Study**

Please visit [this link](https://libguides.princeton.edu/eventstudy) from Princeton University for a detailed explanation of the model using Stata.<br>
[https://libguides.princeton.edu/eventstudy](https://libguides.princeton.edu/eventstudy)

### **Data Preparation and Calculating the Event Window**
Cleaning the Data was pretty straightforward. Because I already had methods to extract rows of company financials based on the amount of years from IPO, all I really had to do in this step was to create a new data table for each company ranging from three years before IPO to five years after. The amount of datapoints ranges from company to company.

In [ ]:
## CALCULATING THE EVENT WINDOW

# break down information on companies on a yearly basis trusting for the moment that this works
COMPANY_FINANCIALS_BY_IPO_DELTA = data_time_horizon(data_ps, [x for x in range(-3, 6)])

for i in range(len(COMPANY_FINANCIALS_BY_IPO_DELTA)):
    frame = COMPANY_FINANCIALS_BY_IPO_DELTA[i]
    frame = frame.query("PE==1")
    COMPANY_FINANCIALS_BY_IPO_DELTA[i] = frame

#Extracts company data from the COMPANY_FINANCIALS_BY_IPO_DELTA table
def get_company_deltas(company_name:str):
    first_entry = COMPANY_FINANCIALS_BY_IPO_DELTA[0]
    company_deltas = first_entry[first_entry["Company"] == company_name]
    for set in COMPANY_FINANCIALS_BY_IPO_DELTA[1:]:
        year_data = set[set["Company"] == company_name]
        company_deltas = pd.concat([company_deltas, year_data], axis=0)
    return company_deltas


To verify that the model works, I first test the analysis one one company in particular.

The company, GoDaddy, was chosen at random.

In [ ]:
# Example to show that it works for 1 company -- to be extracted later
go_daddy_data = get_company_deltas("GoDaddy")

### **Estimating Normal Performance**

To initially build the models, I first find the estimation (training data) and event (testing data) windows for each of the companies tested. For each of the companies, I train a new model for each of the growth variables that are tested.

**estimation window**: financial data for the company from years -3 to 0 from IPO <br>
**event window**: financial data for the company from years 1 to 5 from IPO


In [ ]:
#Returns a boolean to see if the specified financial year is in the event window
def in_event_window(company, row):
    return data_ps[data_ps['Company'] == company]["Deal Year"] < row['Fin Year']

#Returns a boolean to see if the specified financial year is in the estimation window
def in_estimation_window(company, row):
    return ~in_event_window(company, row)

#Later to be abstracted to work for all companies
#Returns a DataFrame of company financials that are in the estimation window
def estimation_window(company_data:pd.DataFrame)->pd.DataFrame:
    return company_data.apply(lambda row:in_event_window('GoDaddy', row), axis=1)


In [ ]:
# Years less than or equal to the IPO. These will be what use to calculate normal performance
estimation_window = go_daddy_data.apply(lambda row: in_estimation_window('GoDaddy', row), axis=1).iloc[:,0]
estimates = go_daddy_data[estimation_window]

# Financials for all years after the IPO that we want to analyze the normal performance with
event_window = estimation_window = go_daddy_data.apply(lambda row: in_event_window('GoDaddy', row), axis=1).iloc[:,0]
events = go_daddy_data[event_window]

def estimation_and_event_windows(company_data, company):
    estimation_window = company_data.apply(lambda row: in_estimation_window(company, row), axis=1).iloc[:,0]
    estimates = company_data[estimation_window]

    event_window = estimation_window = company_data.apply(lambda row: in_event_window(company, row), axis=1).iloc[:,0]
    events = company_data[event_window]

    return estimates, events

Once we have the proper tuples of data, we can now run a regression on the estimation window to see what normal performance was and then save the alphas. For now, using dummy cols which I think could be roughly correlated.

In [ ]:
REGRESSOR_COLS = ['    Gross Profit', '    Diluted EPS', '    Cash Dividends Paid', '    Debt to Equity', '    Stock Price', '    Price to Tangible Book Value']
regressors = estimates[REGRESSOR_COLS]

<b/>Now I fit the model to EBITDA Growth to see how it is impacted</b>

EBITDA Growth is used an example here to make sure that the models works for GoDaddy. The analysis is later generalized to work for all of the companies and growth metrics tested.

In [ ]:
# Creating sm OLS models for each of the variables that I intend to analyze
# The OLS models are contained in a dictionary mapped to the analyzed variable's name
vars_to_analyze = ['roe_growth','revenue_growth','ebitda_growth','net_income_growth','liquidity_growth']

# Returns a dictionary of models for the company estimates passed in
def create_company_models(estimates, regressors, vars_to_analyze):
    models = {}
    
    for var in vars_to_analyze:
        growth = estimates[var]
        Y = growth
        # display(regressors)
        X = sm.add_constant(regressors, has_constant='add')
        # display(X.columns)
        model = sm.OLS(Y, X)
        results = model.fit()
        models[var] = results
    return models

go_daddy_models = create_company_models(estimates, regressors, vars_to_analyze)
go_daddy_models

### **Testing for Abnormal Returns**

Now we are able to test for significance in the testing window. To do this, we must add up the difference between what the model predicted and what the actual growth was for the given year. After the cumulative aggregate growth of the model is found, we can build a simple t-tests to test how closely the difference for each of the years for each of the models is close to 0. By doing so, we can see how far the companies deviate after IPO from the growth rates that were expected of them.

In [ ]:
def cumulative_abnormal_return(models, events, vars_to_analyze, regressor_cols):
    abn_returns = {}
    reg = regressor_cols[:]
    reg.extend(vars_to_analyze)
    event = events[reg]
    reg = ['const'] + regressor_cols[:] 
    for var in vars_to_analyze:
        abn_list = []
        if var in models:
            sample = sm.add_constant(event, has_constant='add')
            abn = models.get(var).predict(sample[reg])
            abn = event[var] - abn
            abn_returns[var] = abn.tolist()
    return abn_returns

all_abnormal_returns = cumulative_abnormal_return(go_daddy_models, events, vars_to_analyze, REGRESSOR_COLS)

In [ ]:
def simple_sig_test_estimates(predicted_var, abnormal_returns):
    simple_sig_test = (sum(abnormal_returns)/len(abnormal_returns))/(stdev(abnormal_returns)/(len(abnormal_returns)**.5))
    return simple_sig_test

def compute_company_estimates(all_abnormal_returns):
    estimates = {}
    for k, v in all_abnormal_returns.items():
        if (len(v) > 2) and not all([x==0 for x in v]):
            estimates[k] = simple_sig_test_estimates(k, v)
    return estimates

### **Generalizing Framework to Work for Multiple Companies**

After testing the model on GoDaddy, we can abstract the model to work for all of the other portfolio companies that should be tested. Below is a dataframe describing the columns within the dataset that will be used to construct the models for each of the companies.

In [ ]:
#COMPANY_FINANCIALS_BY_IPO_DELTA: variable includes financials of all companies and orders them on the -3 to 5 year time horizon specified
#Dataset for the time being does not actually include information on industry data
company_industry_data = deal_info[["Company", "Primary Industry Sector"]]
industry_map = {}
financials_with_industry_data = []
for _, row in company_industry_data.iterrows():
    if row['Company'] not in industry_map:
        industry_map[row['Company']] = row["Primary Industry Sector"]

# populates the dataset with Industry data so I can break down my results based on that
for df in COMPANY_FINANCIALS_BY_IPO_DELTA:
    df['Industry'] = df['Company']
    df['Industry'] = df['Industry'].replace(industry_map)

In [ ]:
COMPANY_FINANCIALS_BY_IPO_DELTA[0].columns
# company_industry_data
COMPANY_FINANCIALS_BY_IPO_DELTA[0].describe().T

Get event study estimate takes in a list of companies and does 2 things

1) It calculates t-statistics for each of the companies given their cumulative abnormal returns
2) maps the calculated statistics for each of the growth metrics to the company name that produced the metric

In [ ]:
# Compute all available significance for the companies that there is enough information for
# Also provides mapping of company significance to outlier data ASSUMES THAT NO SIGNIFICANCE IS EXACTLY EQUAL TO ANOTHER
companies_with_sufficient_data = COMPANY_FINANCIALS_BY_IPO_DELTA[0]['Company'].unique()
REGRESSOR_COLS = ['    Gross Profit', '    Diluted EPS', '    Cash Dividends Paid', '    Debt to Equity', '    Stock Price', '    Price to Tangible Book Value']
vars_to_analyze = ['roe_growth','revenue_growth','ebitda_growth','net_income_growth','liquidity_growth']
def get_event_study_estimates(companies:list[str]):
    company_significance = []
    mp = collections.defaultdict(dict)

    def add_to_mp(company, sig):
        for k, v in sig.items():
            mp[k][v] = company
    
    for c in companies:
        company_deltas = get_company_deltas(c)
        estimates, events = estimation_and_event_windows(company_deltas, c)
        models = create_company_models(estimates, estimates[REGRESSOR_COLS], vars_to_analyze)
        abnormal_returns = cumulative_abnormal_return(models, events, vars_to_analyze, REGRESSOR_COLS)
        significance = compute_company_estimates(abnormal_returns)
        
        company_significance.append(significance)
        add_to_mp(c, significance)

    return mp, company_significance

company_sig_map, all_company_signicance = get_event_study_estimates(companies_with_sufficient_data)
# display(company_sig_map) # display the map that maps t-stats per growth metric to the company that produced it
# display(all_company_signicance) # display the map that shows a list of al computer t-stats for each of the growth metrics

### **Understanding the Analysis**
At this point we have data for the companies tested to see if their growth was able to get sustained into their IPO. Using the data, we can graph the distributions of all the outputted t-tests for the companies and run a separate t-test for each of the growth metrics we want to look at.

In [ ]:
# First thing is to compile each one of the sigificance variables into their proper lists to compare

def list_of_significances(significances):
    mp = collections.defaultdict(list)
    for m in significances:
        for k, v in m.items():
            mp[k] += [v]
    return mp

In [ ]:
# When we have the proper lists, we can then plot histograms for each element
def plot_histograms(mapped_significance):
    for k, v in mapped_significance.items():
        hist = sns.histplot(x=v)
        hist.set(xlabel=(k + " t-test plot"))
        plt.show()

plot_histograms(significance_map)

### **Rerunning the T-Test on Compiled Company Significances**

Lastly, t-statistics for each of the companies is compiled into another t-test so that we may generalize the results of the analysis for portfolio companies as a whole.

In [ ]:
def display_compiled_ttest(key_map):
    for k, v in key_map.items():
        X = v
        Y = 0
        print(f"{k} two-sided ttest")
        display(pg.ttest(X, Y))

display_compiled_ttest(significance_map)    

### **Finding Outliers in the Models and Retesting**

Lastly, we investigate the outliers that are apparent in the event study. The analysis sees two in particular with the data set that was tested (namely The Habit Burger and K2M).

To see what is happening, I look into the event and estimation directly and see what the data looks like. Then I retest the model with updated covariates to see if the significances remain the same. As can be seen, the t-stats decrease drastically.

Please uncomment the function call lines one at a time to see how the outliers are found and how their models change after we slightly modify the training data to use less regressor columns.

In [ ]:
# Returns the list of all outliers for each growth metric given the list of t-statistics
def get_outliers(significances:dict, company_mappings:dict)->dict:
    mp = collections.defaultdict(list)
    def calculate_outliers(data):
        _75th = np.percentile(data,75)
        _25th = np.percentile(data, 25)
        iqr = _75th - _25th
        # print(_75th, _25th)

        return _25th - (1.5*iqr), _75th + (1.5*iqr)

    for k, v in significances.items():
        lower, upper = calculate_outliers(v)

        outliers = np.array([[company_mappings[k][x], round(x,3)] for x in v if (x > upper or x < lower)])
        mp[k] = outliers
    return mp


significance_map = list_of_significances(all_company_signicance)
outliers = get_outliers(significance_map, company_sig_map) 


# Prints out the outliers per growth metric and their t-statistics
def print_outliers(outliers):
    for k, v in outliers.items():
        companies = v[0:, 0]
        sigs = v[0:,1]
        dict = {'companies': companies, 'significance': sigs}
        
        df = pd.DataFrame(data=dict)
        print(k)
        display(df)
        
# print_outliers(outliers) # Run to print the full list of outliers in the model


# Prints the event and estimation window of the outlier companies
def print_outlier_financials(companies:list[str])->None: 
    REGRESSOR_COLS = ['Company', '    Gross Profit', '    Diluted EPS', '    Cash Dividends Paid', 
                      '    Debt to Equity', '    Stock Price', '    Price to Tangible Book Value']
    
    for c in companies:
        df = get_company_deltas(c)
        est, ev = estimation_and_event_windows(df, c)
        display(est[REGRESSOR_COLS])
        display(ev[REGRESSOR_COLS])


# print_outlier_financials(companies) # Run to print the financial data of the outlier companies


REGRESSOR_COLS_RETEST = ['    Gross Profit', '    Diluted EPS', '    Stock Price', '    Price to Tangible Book Value']
var_to_analyze = ['roe_growth','revenue_growth','ebitda_growth','net_income_growth','liquidity_growth']
companies = ['The Habit Burger Grill', 'K2M']


# Recaulculates the event study estimates using updated regressors
def get_event_study_estimate_for_outliers(companies:list[str], REGRESSOR_COLS_RETEST:list[str])->list[float]:
    company_significance = []
    
    for c in companies:
        company_deltas = get_company_deltas(c)
        estimates, events = estimation_and_event_windows(company_deltas, c)
        display(estimates[REGRESSOR_COLS_RETEST])
        models = create_company_models(estimates, estimates[REGRESSOR_COLS_RETEST], var_to_analyze)
        abnormal_returns = cumulative_abnormal_return(models, events, var_to_analyze, REGRESSOR_COLS_RETEST)
        display(abnormal_returns)
        significance = compute_company_estimates(abnormal_returns)
        
        company_significance.append(significance)
    
    return company_significance

company_estimates = get_event_study_estimate_for_outliers(companies, REGRESSOR_COLS_RETEST)
display(company_estimates)

<br>
<br>
<br>

**Click [here](#top) to jump to the top of the notebook